# reference

http://lamp.epfl.ch/files/content/sites/lamp/files/teaching/progfun/ScalacheckTutorial.html

7장에서 병렬 계산의 표현을 위한 함수적 라이브러리를 설계했다. 이 과정에서 API가 자료 형식들과 그 형식들에 대한 함수들, 그리고 중요하게는 그런 함수들 사이의 관계를 나타내는 *법칙* 또는 *속성*들로 이루어진 하나의 대수를 형성해야 한다는 개념을 소개했다.

이번 장에서는 *속성 기반 검사(property based testing)*를 위한 간단하지만 강력한 라이브러리를 만들어 본다.

# 8.1 속성 기반 검사의 간략한 소개

In [ ]:
val intList = Gen.listOf(Gen.choose(0, 100))
val prop = 
    forAll(intList)(ns => ns.reverse.reverse == ns) &&
    forAll(iniList)(ns => ns.headOption == ns.reverse.lastOption)
val failingProp = forAll(intList)(ns => ns.reverse == ns)

In [ ]:
scala> prop.check
+ OK, passed 100 tests.

scala> failingProp.check
! Falsified after 6 passed tests
> ARG_0: List(0, 1)

여기서 initList의 의미는 Gen[List[Int]]

<img src="./figures/genandproperty.png">

prop.check가 testcase를 생성해서 실행해 준다.

* test case minimization: 검사가 실패하면 프로임워크는 검사에 실패하는 가장 작은 검례에 도달할 때까지 더 작은 검례들을 시도한다. 예를 들어 어떤 속성이 크기가 10인 목록에 대해 실패한다면, 프레임워크는 그 검사에 실패하는 가장 작은 목록에 도달할 때까지 점점 더 작은 목록으로 검사를 수행하고 찾아낸 최소의 목록을 보고한다. 그런 최소의 검례는 디버깅에 도움이 된다.

* exhaustive test case generation: Gen[A]가 생성할 수 있는 모든 값의 집합을 정의역이라고 부른다. 정의역이 충분히 작다면(이를테면 100 미만의 짝수 정수 등) 표본 값들을 생성하는 대신 정의역의 모든 값을 검사할 수 있다.(전수 검사) 만일 정의역의 모든 값에 대해 속성이 성립한다면, 단지 반례가 없음을 확인한 차원을 넘어서 속성ㅇ이 실제로 증명된 것이다.

https://en.wikipedia.org/wiki/Domain_of_a_function

ScalaCheck는 여러 속성 기반 검사 라이브럴리 중 하나일 뿐이다. ScalaCheck에 딱히 문제가 있는 것은 아니지만, 이번 장에서는 속성 ㄱ반 검사 라이브러리를 직접 만들어 보기로 하겠다.

# 8.2 자료 형식과 함수의 선택

이번엔 속성 기반 검사를 위한 라이브러리다. 7장에서 처럼 어깨너머로 배우게 될 것이다. 속성 기반 검사에 익숙하지 않으면 좋겠다. 그러면 새로운 영역과 설계 공간을 탐험하면서 스스로 뭔가를 발견해 낼 기회가 될 것이다.

## 8.2.1 API의 초기 버전

In [ ]:
val intList = Gen.listOf(Gen.choose(0,100))
val prop = 
    forAll(intList)(ns => ns.reverse.reverse == ns) &&
    forAll(intList)(ns => ns.headOption == ns.reverse.lastOption)

type을 generic하게 만들 수 있지 않을까?

In [ ]:
def listOf[A](a: Gen[A]): Gen[List[A]]

뭘 알 수 있지? 생성기에 크기를 주어지지 않았다.

크기를 알려줄 수단이 필요하지 않을까? 그렇다.

그런데 특정 크기를 가정하는 건 유연성이 떨어지지 않을까? 맞다.

In [ ]:
def listOfN[A](n: Int, a: Gen[A]): Gen[List[A]]

이것이 좋아 보이는가? 나름 좋아 보이지만 암묵적인 생성기도 필요해 보인다.

## 8.2.2 속성의 의미와 API

Prop의 속성들에는

* forAll:
* &&:
* check:

이 있다.

ScalaCheck에서 check는 console 출력이라는 side effect가 있다.

In [ ]:
trait Prop {
    def check: Unit
    def &&(p: Prop): Prop = ???
}

&&를 구현한느 유일한 선택은 두 check 메소드를 모두 실행하는 것이다. check가 검사 결과 메시지를 출력한다면 &&에 의해 두 개의 메시지가 출력되는데 그 메시지들은 두 속성의 성공 실패 여부를 각자 따로 판정한 결과이다.

check가 정보를 정보를 *폐기*하는게 문제이다.

In [ ]:
object Prop {
    type SuccessCount = Int
    ...
}

trait Prop {
    def check: Either[???, SuccessCount]
}

실패하는 경우 뭘 돌려 줘야 하나? 아직 test case의 반환에 대해 정해진 건 없다. Prop[A]형식으로 만들어야 하나? Either[A, SuccessCount]로 반환하면 되나? 그 형식은 실패했을 때나 계산을 계속할 때 모두 중요하다.

검사의 목적은 버그를 찾아내고 그 버그를 드러내게 한 검례가 무엇인지 제시해서 프로그래머가 버그를 찾고 고치게 만드는 것일 뿐이다.

In [ ]:
object Prop {
    type FailedCase = String
    type SuccessCount = Int
}

trait Prop {
    def check: Either[[FailedCase, SuccessCount], SuccessCount]
}

check가 실패했을 때 Left((s, n))을 반환한다. s는 속성이 실패한 값을 n은 성공한 test case의 개수

In [ ]:
def forAll[A](a: Gen[A])(f: A => Boolean): Prop

아직 Gen의 구체적 표현은 모른다

## 8.2.3 생성기의 의미와 API

Gen[A]는 A형식의 값을 무작위로 생성하는 것이다. 이는 6장의 RNG를 사용한다. 이와 비슷하게 Gen을 난수 발생기의 State 전이를 감싸는 하나의 형식으로 만들 수도 있다.

In [ ]:
case class Gen[A](sample: State[RNG, A])

In [ ]:
def unit[A](a: => A): Gen[A]
def boolean: Gen[Boolean]
def listOfN[A](n: Int, g: Gen[A]): Gen[List[A]]

7장에서 어떤 연산들이 *기본수단*이고 어떤 연산ㄷㄹ이 그로부터 *파생된* 연산들인지 이해하고 작지만 표현력 있는 기본 수단들의 집합을 구하는데 관심이 있다.

* 어떤 구간(범위) 안에 Int 값 하나를 생성하는 기본 수단이 있다고 할 떄, 어떤 구간 안의 (Int, Int) 쌍을 생성하기 위해 새로운 기본 수간이 필요할까?
* Gen[A]에서 Gen[Optoin[A]]를 얻을 수 있을까? Gen[Option[A]]에서 Gen[A]를 얻는 것은 어떨까?
* 기존의 기본 수단들을 이용해서 어떤 방식으로든 문자열을 생성할 수 있을까?

## 8.2.4 생성된 값들에 의존하는 생성기

둘째 문자열이 첫문자열의 문자들로만 이루어진다는 조건을 만족하는 문자열 쌍 생성기 Gen[(String, String)]이 있다고 하자. Gen[Int]로 0에서 11 사이의 정수를 선택하는 Gen[List[Double]]이 그 값을 목록의 길이로 사용해서 목록을 생성한다고 하자. 한 생성기가 만들어지고 나머지는 다른 생성할 값들을 결정하는데 쓰인다. 이런 방식을 지원하려면 flatMap이 필요하다.

## 8.2.5 Prop 자료 형식의 정련

In [ ]:
trait {
    def check: Either[(FailedCase, SuccessCount), SuccessCount]
}

요건 걍 Either일 뿐 몇가지 정보가 빠져 있다. 성공수는 있지만 test case가 성공했을 때 test case의 갯수는 없다. 이 수는 추상화하여 넣는 것이 좋다.

In [ ]:
type TestCases = Int
type Result = Either[(FailedCase, SuccessCount), SuccessCount]
case class Prop(run: TestCases => Result)

현재는 성공수를 Either의 양쪽에 모두 넣지만 성공했을 때 갯수는 미리 알고 있다.

In [ ]:
type Result = Option[(FailedCase, SuccessCount)]
case class Prop(run: TestCases => Result)

None은 성공을 Option은 실패를 뜻한다. 하지만 의도가 반대로 되어 있다. 새롭게 만든다면...

In [ ]:
sealed trait Result {
    def isFalsified: Boolean
}
case object Passed extends Result {
    def isFalsified = false
}
case class Falsified(failure: FailedCase, successes: SuccessCount) extends 
    Result {
        def isFalsified = true
        
}

여기에 맞춰 forAll도 다시 정의해 보자

In [ ]:
def forAll[A](a: Gen[A])(f: A => Boolean): Prop

아직 forAll은 Prop을 돌려 주기에 정보가 부족하다. Prop.run에는 시도할 검례 갯수 외에도 검례들을 생성하는데 피룡한 정보가 필요하다. Prop.run이 Gen의 현재 표현을 이용해서 무작위 검례들을 생성해야 한다면 RNG가 필요하다.

In [ ]:
case class Prop(run: (TestCasses, RNG) => Result)

In [ ]:
def forAll[A](as: Gen[A])(f: A => Boolean): Prop = Prop {
    (n, rng) => randomStream(as)(rng).zip(Stream.from(0)).take(n).map{
        case (a, i) => try{
            if(f(a)) Passed else Falsified(a.toString, i)
        } catch {
            case e: Exception => Falsified(buildMsg(a, e), i)
        }
    }.find(_.isFalsified).getOrElse(Passed)
}

def randomStream[A](g: Gen[A])(rng: RNG): Stream[A] =
    Stream.unfold(rng)(rng => Some(g.sample.run(rng)))

def buildMsg[A](s: A, e: Exception): String = 
    s"test case: $s\n" +
    s"generated an exception: ${e.getMessage}\n" +
    s"stack trace:\n ${e.getStackTrace.mkString("\n")}"

# 8.3 검례 최소화

test case 최소화는 해당 test case가 실패하게 만드는 가장 간단한 test case를 찾는 것이다. 이것은 debugging에 도움을 준다.

* shrinking(수축) - 실패한 검례가 나왔다면 개별적인 절차를 띄워서 그 test case의 크기를 점점 줄여가면서 검사를 반복하되 검사가 더 이상 실패하지 않으면 멈춘다. 이러한 최소화 공정을 수축이라고 부른
* sized generation(크기별 생성) - 검사 실패 후에 검례들을 수축하는 대신, 애초에 크기와 복잡도를 점차 늘려가면서 test case을 생성한다. 이런 착안을 검사 실행기가 적용 가능한 크기들의 공간을 크게 건너 뛰면서도 여전히 가장 작은 실패 test case를 찾을 수 있도록 확장하는 방법도 여러 가지이다.

ScalaCheck(와 QuickCheck)는 첫번째, 우리는 두번째 방법을 시도한다.

In [ ]:
case class SGen[+A](forSize: Int => Gen[A])

In [ ]:
def forAll[A](g: SGen[A])(f: A => Boolean): Prop

forAll은 구현할 수 없다. 크기가 없기 때문읻.

In [ ]:
type MaxSize = Int
case class Prop(run: (MaxSize, TestCases, RNG) => Result)

def forAll[A](g: SGen[A])(f: A => Boolean): Prop = 
    forAll(g(_))(f)

def forAll[A](g: Int => Gen[A])(f: A => Boolean): Prop = Prop {
    (max, n, rng) =>
    val casesPerSize = (n + (max - 1)) / max
    val props: Stream[Prop] = 
        Stream.from(0).take((n min max) + 1).map(i => forAll(g(i))(f))
    val prop: Prop = 
        props.map(p => Prop{ (max, _, rng) =>
            p.run(max, casesPerSize, rng)
        }).toList.reduce(_ && _)
    prop.run(max, n, rng)
}

# 8.4 라이브러리의 사용과 사용성 개선

## 8.4.1 간단한 예제 몇 가지

In [ ]:
val smallInt = Gen.choose(-10, 10)
val maxProp = forAll(listOf(smallInt)) { ns =>
    val max = ns.max
    !ns.exists(_ > max)
}

Prop의 run을 돌리고 결과를 콘솔에 보여주는 보조 함수를 도입하면 좋을 듯

In [ ]:
def run(p: Prop, 
        maxSize: Int = 100, 
        testCases: Int = 100, 
        run: RNG = RNG.Simple(System.currentTimeMilis)): Unit = {
    p.run(maxSize, testCases, rng) match {
        case Falsified(msg, n) =>
            println(s"! Falsified after $n passed tests:\n $msg")
        case Passed =>
            println(s"+ OK, passed $testCases tests.")
    }
}

## 8.4.2 병렬 계산을 위한 검사 모음 작성

7장에서 병렬 계산에 대해 반드시 성립해야 하는 법칙들을 밝혀냈었다.(?) 7장에서 살펴본 첫 법칙은 사실 특정한 test case 였다.

In [ ]:
map(unit(1))(_ + 1) == unit(2)

이를 표현하는 것은 물론 가능하지만, 아주 깔끔하게 표현하지는 못한다.

In [ ]:
val ES: ExecutorService = Executors.newCachedThreadPool
val p1 = Prop.forAll(Gen.unit(Par.unit(1)))(i =>
    Par.map(i)(_ + 1)(ES).get == Par.unit(2)(ES).get
    )

이 법칙을 증명한다.

In [ ]:
def check(p: => Boolean): Prop

In [ ]:
def check(p: => Boolean): Prop = {
    lazy val result = p
    forAll(unit(()))(_ => result)
}

check는 memoization하더라도 여러번 실행하게 된다. run(check(true))를 하다라도 true에 대해서 100번을 수행하고 "OK, passed 100 tests."를 출력할 것이다.

현재 Prop의 표현은 그냥 (MaxSize, TestCases, RNG) => Result 형식의 함수일 뿐임을 기억하자. 여기서 Result는 Passed나 Falsified이다. 갯수를 무시하는 Prop을 구축하는 check라는 기본 수단을 새로 만들어 보자

In [ ]:
def check(p: => Boolean): Prop  = Prop{ (_,_,_) =>
    if(p) Passed else Falsified("()",0)
}

이것은 다음과 같은 새로운 Result의 도입이 필요하다.

In [ ]:
case object Proved extends Result

이제 Passed대신 Proved를 반환하면 된다.

In [ ]:
def run(p: Prop,
       maxSize: Int = 100,
       testCases: Int = 100,
       rng: RNG = RNG.Simple(System.currentTimeMillis)): Unit = {
    p.run(maxSize, testCases, rng) match {
        case Falsified((msg, n)) =>
            println(s"! Falsified after $n passed tests:\n $msg")
        case Passed =>
            println(s"+ OK, passed $testCases tests.")
        case Proved =>
            println(s"+ OK, proved property.")
    }
}

&&같은 Prop의 조합기들의 구조도 적절히 수정되어야 한다.

In [ ]:
val p2 = Prop.check {
    val p = Par.map(Par.unit(1))(_ + 1)
    val p2 = Par.unit(2)
    p(ES).get == p2(ES).get
}

이제 명확하다. Par를 검증하는데 내부의 구현이 드러나 버렸다. 이를 없애려면 map2를 이용해서 상등 비교를 Par로 승급시키는 것이다.

In [ ]:
def equal[A](p: Par[A], p2: Par[A]): Par[Boolean] = 
    Par.map2(p, p2)(_ == _)

val p3 = check {
    equal(
        Par.map(Par.unit(1))(_ + 1),
        Par.unit(2)
    )
}

양변을 개별적으로 실행하는 것에 비하면 나아졌다.

In [ ]:
val S = weighted(
    choose(1,4).map(Executors.newFixedThreadPool) -> .75,
    unit(Executors.newCachedThreadPool) -> .25
)

def forAllPar[A](g: Gen[A])(f: A => Par[Boolean]): Prop = 
    forAll(S.map2(g)((_,_))) { case (s, a) => f(a)(s).get }

S.map2(g)((_,_))라는 구문에는 출력들의 쌍을 생성하기 위해 두 생성기를 조합한다는 의도에 비해 잡음이 많은 편이다. 간결하게 하자.

In [ ]:
def **[B](g: Gen[B]): Gen[(A, B)] = (this map2 g)((_,_))

def forAllPar[A](g: Gen[A])(f: A => Par[Boolean]): Prop =
    forAll(S ** g) { case (s, a) => f(a)(s).get }

여러 생성기를 튜플로 엮을 때 잘 동작한다. \*\*를 패턴으로 사용할 수 있도록 unapply 함수가 있는 \*\*라는 이름의 객체를 정의한다면!

In [ ]:
object ** {
    def unapply[A, B](p: (A, B)) = Some(p)
}

이제 s는 스레드가 1~4개인 고정 크기 스레드 풀들을 가변적으로 사용하는 크기가 한정되지 않은 스레드 풀도 고려하는 Gen[ExecutorService]이다. 깔끔하지 않은가?

In [ ]:
val p2 = checkPar {
    equal(
        Par.map(Par.unit(1))(_ + 1),
        Par.unit(2)
    )
}

7장의 속성도 살펴보면

In [ ]:
map(unit(x))(f) == unit(f(x))
map(y)(x => x) == y

In [ ]:
val pint = Gen.choose(0,10) map (Par.unit(_))
val p4 = forAllPar(pint)(n => equal(Par.map(n)(y => y), n))

map에 실제로 영향을 미치는 것은 병렬 계산 구조이다.

# 8.5 고차 함수의 검사와 향 후 개선 방향

지금까지 표현력이 상당히 좋은 속성 기반 검사 라이브러리를 만들었다. 그런데 간과한 부분이 하나 있다. 현재 라이브러리는 고차 함수를 검사하기에 적합한 수단을 제공하지 않는다. 생성기들을 이용해서 **자료**를 생성하는 수단은 많이 갖추었지만 **함수**를 생성하는 적당한 수단은 없다.

In [ ]:
val isEven = (i: Int) => i%2 == 0
val takeWhileProp = 
    Prop.forAll(Gen.listOf(int))
        (ns => ns.takeWhile(isEven).forAll(isEven))

이 경우 takeWhile이 자동으로 생성해 준다면 더 좋겠다. Gen[Int]에 대해서 Gen[String => Int]를 생성하자고 하자.

In [ ]:
def genStringIntFn(g: Gen[Int]): Gen[String => Int] = 
    g map (i => (s => i))

이걸로는 부족하단다. 이는 단순히 constant function일 뿐인데 takeWhile의 검사를 위해서는 Boolean을 돌려 주는 함수가 필요한데 이 생성기가 만드는 함수는 항상 true를 돌려주거나 항상 false를 돌려준다.

# 8.6 생성기의 법칙들

Gen형식을 구현한 함수중에는 Par나 List, Stream, Option에 대해 정의한 다른 함수들과 상당히 비슷해 보이는 것들이 많다는 점에 주목한 독자가 있을 것이다.

아래는 Par에 정의한 map

In [ ]:
def map[A, B](a: Par[A])(f: A => B): Par[B]

아래는 Gen에서 정의한 map

In [ ]:
def map[B](f: A => B): Gen[B]

Option, List, Stream, State에도 이와 비슷한 모습의 함수가 정의되어 있다.

map(x)(id) == x

Gen.map에서도 동일하게 성립할까? Stream, List, Option, State는 어떨까? 실제 모두 성립힌다. 함 해 보시라.

# 8.7 요약

속성 기반 검사 영역의 예를 통해서 함수적 라이브러리 설계 연습을 계속 이어나갔다.

속성 기반 검사를 배우는 것은 목적이 아니라 함수적 설계의 특정 측면들을 강조한 것이 목적이다. 추상적인 대수와 구체적인 표현을 오가면서 대수에서 얻은 지식을 표현에 적용하고 표현에서 얻은 지식을 대수에 적용했다.

map, flatMap과 같은 조합기를 만났다. 이들은 서명만 비슷한게 아니라 법칙들도 비슷하다. 소프트웨어 세계에서는 겉으로 보기에는 서로 다른 *문제점*들이 대단히 많지만 그에 대한 함수적 *해법*들의 공간은 의외로 훨씬 적다. 다종 다양한 문제 영역들에서 계속해서 나타나는 몇몇 근본적인 구조들의 조합일 뿐인 라이브러리들도 많다. 이는 3부에서 살펴볼 코드 재사용의 기회이다.

다음장에 *파싱*을 살펴 본다. 파싱에도 나름의 난제들이 존재한다. 다음 장에서는 지금까지와는 약간 다른 접근 방법을 사용하지만, 그래도 익숙한 패턴들이 드러날 것이다.

# Thank you so much

# Q & A